In [ ]:
from matplotlib import pyplot as plt
from datetime import datetime
import math
import sys
from pprint import pprint
from datetime import datetime

sys.path.append('../..')

In [2]:
# from model.amm.omnipool_amm import OmnipoolState, DynamicFee
from model.amm.agents import Agent
from model.amm.trade_strategies import general_arbitrage, invest_all
from model.amm.global_state import GlobalState
from model.amm.stableswap_amm import StableSwapPoolState
from model.amm.exchange import Exchange

class CEXDummy(Exchange):
    def __init__(tokens: dict[str: float]):
        self.prices = tokens
        self.liquidity = {tkn: 0 for tkn in tokens}
    
    def price(tkn):
        return self.prices[tkn]
    
    def buy_spot(tkn_buy, tkn_sell):
        return self.price(tkn_buy) / self.price(tkn_sell)
    
    def sell_spot(tkn_sell, tkn_buy):
        return self.price(tkn_sell) / self.price(tkn_buy)
    
    def swap(
        agent: Agent, 
        tkn_buy: str, 
        tkn_sell: str,
        buy_quantity: float,
        sell_quantity: float
    ):
        if buy_quantity:
            sell_quantity = self.calculate_sell_from_buy(tkn_buy, tkn_sell, buy_quantity)
        elif sell_quantity:
            buy_quantity = self.calculate_buy_from_sell(tkn_buy, tkn_sell, sell_quantity)
            
        if tkn_buy not in agent.holdings:
            agent.holdings[tkn_buy] = 0
            
        agent.holdings[tkn_sell] -= sell_quantity
        agent.holdings[tkn_buy] += buy_quantity
        self.liquidity[tkn_sell] += sell_quantity
        self.liquidity[tkn_buy] -= buy_quantity
        
    def calculate_buy_from_sell(tkn_buy, tkn_sell, sell_quantity):
        return sell_quantity * self.price(tkn_sell) / self.price(tkn_buy)
    
    def calculate_sell_from_buy(tkn_sell, tkn_buy, buy_quantity):
        buy_quantity * self.price(tkn_buy) / self.price(tkn_sell)

rebase_amount = 1000        
def rebase(tkn: str):
    def evolve(state: GlobalState):
        tkn_holders = {
            pool_id: pool.liquidity[tkn]
            for pool_id, pool in state.pools.items()
        }
        tkn_holders.update({
            agent_id: agent.holdings[tkn]
            for agent_id, agent in state.agents.items()
        })
        total_liquidity = sum(tkn_holders.values())
        awards = 0
        for holder in tkn_holders:
            if isinstance(holder, Agent):
                awards += rebase_amount * holder.holdings[tkn] / total_liquidity
                holder.holdings[tkn] += rebase_amount * holder.holdings[tkn] / total_liquidity
            elif isinstance(holder, Exchange):
                awards += rebase_amount * holder.liquidity[tkn] / total_liquidity
                holder.liquidity[tkn] += rebase_amount * holder.liquidity[tkn] / total_liquidity
        if (state.time_step % 100 == 0):
            print(awards)
        return state
    return evolve

binance = CEXDummy({'S1': 1, 'S2': 1})
stableSwap = StableSwapPoolState(
    tokens={'S1': 1000000, 'S2': 1000000},
    amplification=100,
    trade_fee=0.001,
    unique_id='stableswap'
)
LP = Agent(
    holdings={'S1': rebase_amount},
    trade_strategy=invest_all('stableswap'),
    unique_id='LP'
)
non_LP = Agent(
    holdings={'S1': rebase_amount},
    trade_strategy=None,
    unique_id='non_LP'
)
        
initial_state = GlobalState(
    pools=[stableswap, binance],
    agents=[LP, non_LP, arbitrageur],
    external_market={'S1': 1, 'S2': 1},
    evolve_function=rebase('S1')
)

ImportError: cannot import name 'general_arbitrage' from 'model.amm.trade_strategies' (C:\Users\jepid\PycharmProjects\HydraDX-simulations\hydradx\notebooks\Omnipool\../..\model\amm\trade_strategies.py)

In [ ]:
events = run.run(initial_state, time_steps=1000)
events = processing.postprocessing(events)

In [ ]:
pu.plot(events, pool='stableswap', prop='liquidity')
pu.plot(events, agent='Arbitrageur', prop='holdings', key=['S1', 'S2'])

In [ ]:
impermanent_loss = [
    (1 - event.pools['stableswap'].cash_out(event.agents['LP'], event.external_market)
    / sum(event.agents['non LP'].holdings.values())) * 100
    for event in events
]
total_supply = [
    sum([
        event.pools['stableswap'].liquidity['S1'], 
        event.agents['Arbitrageur'].holdings['S1'], 
        event.agents['LP'].holdings['S1'], 
        event.agents['non LP'].holdings['S1']
    ]) for event in events
]


In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(total_supply, impermanent_loss)
plt.title

In [ ]:
events[2].agents['LP'].holdings

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(total_supply, [event.pools['stableswap'].cash_out(event.agents['LP'], event.external_market) for event in events])
plt.plot(total_supply, [sum(event.agents['non LP'].holdings.values()) for event in events])

In [ ]:
(
    events[-1].pools['stableswap'].liquidity['S1']
    + events[-1].agents['LP'].holdings['S1']
    + events[-1].agents['non LP'].holdings['S1']
    + events[-1].agents['Arbitrageur'].holdings['S1']
)

In [ ]:
events[-1].agents['non LP'].holdings['S1']